In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
def preprocess(path):
    
    val = pd.read_csv(path)
    
    val.drop(columns=['PassengerId','Name','Ticket','Pclass','Fare'], inplace=True)
    val.replace(['male','female'], [1,0],inplace=True)
    
    embar_val = pd.DataFrame({'S' : val.Embarked == 'S','C' : val.Embarked == 'C','Q' : val.Embarked == 'Q','NAN' : val.Embarked == 'nan'}).astype(int)
    val = val.drop('Embarked',axis=1)
    
    alone = pd.DataFrame({'alone' : val.Parch + val.SibSp == 0}).astype(int)
    #val = val.drop(['Parch','SibSp'],axis=1)
    val = pd.concat([val,embar_val,alone], axis=1, join='inner')
    val['Cabin'].fillna('X', inplace = True)
    val['Cabin_encoded'] = val['Cabin'].apply(lambda x:x[0] if len(x) > 1 else x)
    
    t = pd.DataFrame({'X' : val.Cabin_encoded == 'X','C' : val.Cabin_encoded == 'C','E' : val.Cabin_encoded == 'E','G':val.Cabin_encoded == 'G','D':val.Cabin_encoded == 'D','A': val.Cabin_encoded == 'A','B': val.Cabin_encoded == 'B','F': val.Cabin_encoded == 'F','T':val.Cabin_encoded == 'T'}).astype(int)
    val.Age.fillna(0,inplace=True)
    val = pd.concat([val, t], axis=1, join='inner')
    val = val.drop(['Cabin_encoded','Cabin'],axis=1)
    
    if 'Survived' in val.columns:
        global survive
        survive = val.Survived
        val.drop('Survived', inplace=True, axis=1)
    
    return val

In [3]:
train = preprocess('data/train.csv')
train.head()

test = preprocess('data/test.csv')
y_test = pd.read_csv('data/surv_test.csv').iloc[:,-1]

In [4]:
train_x = train.as_matrix()
train_y = pd.get_dummies(survive).as_matrix()

test_x = test.as_matrix()
test_y = pd.get_dummies(y_test).as_matrix()

/home/gaourab/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/gaourab/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/gaourab/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/home/gaourab/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [5]:
print(np.shape(test_x))
print(np.shape(test_y))

print(np.shape(train_x))
print(np.shape(train_y))

(418, 18)
(418, 2)
(891, 18)
(891, 2)


In [6]:
epoch_n = 1000
class_n = 2
bathc_size = 32
learning_rate = 0.001

x = tf.placeholder(dtype=tf.float32, shape=[None,18])
y = tf.placeholder(dtype=tf.float32, shape=[None,2])

In [7]:
def n_model(x):
    
    layer1 = tf.layers.dense(x, units=100, activation=tf.nn.relu)
    layer2 = tf.layers.dense(layer1, units=200, activation=tf.nn.relu)
    drop = tf.layers.dropout(layer2, rate=0.5)
    out = tf.layers.dense(drop, units=class_n)
    
    return out
    

In [8]:
pred = n_model(x)

loss = tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=pred)
optimize = tf.train.AdamOptimizer(learning_rate).minimize(loss)
corr_pred = tf.equal(tf.argmax(pred,axis=1), tf.argmax(y, axis=1))
acc = tf.reduce_mean(tf.cast(corr_pred, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(epoch_n):
        sess.run(optimize, feed_dict = {x : train_x, y : train_y})
        
        acc_p = sess.run(acc, feed_dict = {x: test_x, y : test_y })
        print(acc_p)
    

0.5598086
0.36842105
0.39473686
0.47368422
0.6650718
0.64593303
0.638756
0.638756
0.6411483
0.6483254
0.5861244
0.46411484
0.46411484
0.59090906
0.65311
0.6363636
0.6363636
0.6363636
0.6363636
0.64593303
0.7200957
0.81100476
0.80861247
0.791866
0.6985646
0.6602871
0.65311
0.6698565
0.7320574
0.8181818
0.78708136
0.8014354
0.77272725
0.7583732
0.7751196
0.78708136
0.784689
0.78229666
0.76555026
0.791866
0.784689
0.77751195
0.77751195
0.78229666
0.7751196
0.7751196
0.7703349
0.77272725
0.77751195
0.76555026
0.7631579
0.7679426
0.7703349
0.7583732
0.76076555
0.76076555
0.7583732
0.7679426
0.7703349
0.77272725
0.78229666
0.78229666
0.78229666
0.78229666
0.784689
0.79425836
0.7894737
0.8181818
0.82535887
0.83253586
0.8779904
0.9019139
0.9019139
0.90909094
0.9066986
0.9114832
0.9138756
0.91626793
0.923445
0.92105263
0.9282297
0.923445
0.9282297
0.92583734
0.9282297
0.930622
0.923445
0.930622
0.9282297
0.930622
0.93301433
0.93301433
0.930622
0.930622
0.9282297
0.9282297
0.9282297
0.92583734
0

0.8492823
0.84689
0.8516746
0.84689
0.85406697
0.8444976
0.8588517
0.84689
0.8564593
0.8444976
0.8564593
0.84689
0.8588517
0.84689
0.8516746
0.84689
0.861244
0.8444976
0.85406697
0.84210527
0.85406697
0.84210527
0.85406697
0.8444976
0.8516746
0.8397129
0.8564593
0.8444976
0.8588517
0.8444976
0.8636364
0.84210527
0.861244
0.8492823
0.8588517
0.84210527
0.861244
0.8397129
0.8684211
0.84689
0.8660287
0.8444976
0.8564593
0.84210527
0.85406697
0.8397129
0.8564593
0.84210527
0.8492823
0.84210527
0.8492823
0.8444976
0.85406697
0.84210527
0.861244
0.84689
0.8636364
0.8492823
0.861244
0.84210527
0.8636364
0.84210527
0.8660287
0.84210527
0.8636364
0.84210527
0.8564593
0.84210527
0.85406697
0.8444976
0.8492823
0.8444976
0.84210527
0.84210527
0.8444976
0.8397129
0.8444976
0.84210527
0.85406697
0.8397129
0.85406697
0.84210527
0.8588517
0.8444976
0.87081337
0.8397129
0.87081337
0.84210527
0.87081337
0.8349282
0.8492823
0.8492823
0.84210527
0.861244
0.8444976
0.8636364
0.84210527
0.861244
0.84210527


In [10]:
"manual weight and bias tutorial : https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/neural_network_raw.py"

'manual weight and bias tutorial : https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/neural_network_raw.py'

In [27]:
hidden_1 = 100
hidden_2 = 200

input_shape = 18
pred_class = 2

x_ = tf.placeholder('float', shape=[None,input_shape])
y_ = tf.placeholder('float', shape=[None,pred_class])

In [28]:
weights = {
    'w1' : tf.Variable(tf.random_normal([input_shape, hidden_1])),
    'w2' : tf.Variable(tf.random_normal([hidden_1, hidden_2])),
    'out': tf.Variable(tf.random_normal([hidden_2, pred_class]))
}

bias = {
    'b1' : tf.Variable(tf.random_normal([hidden_1])),
    'b2' : tf.Variable(tf.random_normal([hidden_2])),
    'out': tf.Variable(tf.random_normal([pred_class]))
}

In [29]:
def neural_model(x):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), bias['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), bias['b2'])
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), bias['out'])
    
    return out_layer

In [30]:
logistic = neural_model(x_)


loss_ = tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logistic)
optimize_ = tf.train.AdamOptimizer(0.001).minimize(loss_)
corr_pred_ = tf.equal(tf.argmax(logistic,axis=1), tf.argmax(y_, axis=1))
acc_ = tf.reduce_mean(tf.cast(corr_pred_, tf.float32))



In [31]:
print(np.shape(test_x))
print(np.shape(test_y))

print(np.shape(train_x))
print(np.shape(train_y))

(418, 18)
(418, 2)
(891, 18)
(891, 2)


In [34]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(1000):
        t = sess.run(optimize_, feed_dict = {x_ : train_x, y_ : train_y})
        
        if i % 50 == 0:
            acc_p = sess.run(acc_, feed_dict = {x_: test_x, y_ : test_y })
            print('Epoch {} accuracy {}'.format(i, acc_p))
    

Epoch 0 accuracy 0.4712918698787689
Epoch 50 accuracy 0.7870813608169556
Epoch 100 accuracy 0.8181818127632141
Epoch 150 accuracy 0.8636363744735718
Epoch 200 accuracy 0.8827751278877258
Epoch 250 accuracy 0.9043062329292297
Epoch 300 accuracy 0.7440191507339478
Epoch 350 accuracy 0.8971291780471802
Epoch 400 accuracy 0.8708133697509766
Epoch 450 accuracy 0.7822966575622559
Epoch 500 accuracy 0.8875598311424255
Epoch 550 accuracy 0.8444976210594177
Epoch 600 accuracy 0.7511961460113525
Epoch 650 accuracy 0.6698564887046814
Epoch 700 accuracy 0.8947368264198303
Epoch 750 accuracy 0.5
Epoch 800 accuracy 0.9258373379707336
Epoch 850 accuracy 0.7248803973197937
Epoch 900 accuracy 0.9138755798339844
Epoch 950 accuracy 0.8684210777282715
